In [ ]:
import jax
import netket as nk
import numpy as np
from netket.operator.spin import sigmax,sigmaz
import time
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
L = 6
N = L*L
hi = nk.hilbert.Spin(s=1 / 2, N =  N)

H = -1*sigmax(hi, 0)*sigmaz(hi, 1)*sigmaz(hi, 2)
H += -1*sigmax(hi, L-1)*sigmaz(hi, L-2)*sigmaz(hi, 2*L-1)
H += -1*sigmax(hi, L*(L-1))*sigmaz(hi, L*(L-1)+1)*sigmaz(hi, L*(L-2))
H += -1*sigmax(hi, L*L-1)*sigmaz(hi, L*L-2)*sigmaz(hi, L*(L-1)-1)
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmaz(hi,i+L)*sigmaz(hi,(i-L)) for i in (np.arange(L,L*(L-1), L))])  #left
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i-1))*sigmaz(hi,i+L)*sigmaz(hi,(i-L)) for i in (np.arange(2*L-1, L*L-1, L ))])  #right
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmaz(hi,i-1)*sigmaz(hi,(i+L)) for i in range(1, L-1)])  #top
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmaz(hi,i-1)*sigmaz(hi,(i-L)) for i in range(L*(L-1)+1, L*L-1)])  #bottom
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmaz(hi,i+2)*sigmaz(hi,(i+L))*sigmaz(hi,i-L) for i in ((np.meshgrid(np.arange(L-2)+1, np.arange(L-2)+1)[1]*L+np.meshgrid(np.arange(L-2)+1, np.arange(L-2)+1)[0]).ravel())])



In [35]:
class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=2*x.shape[-1],
                     use_bias=True,
                     param_dtype=jnp.complex64,
                     kernel_init=nn.initializers.normal(stddev=0.01),
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x
n_iter = 10000
model = FFNN()
ma = nk.models.RBM(alpha=2)
sa = nk.sampler.MetropolisLocal(hilbert=hi)

# Optimizer
warmup_cosine_decay_scheduler = optax.warmup_cosine_decay_schedule(init_value=0.01, peak_value=0.05,
                                                                   warmup_steps=100,
                                                                   decay_steps=,
                                                                   end_value=n_iter)
op = nk.optimizer.Sgd(learning_rate=0.05)
# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.1)

# The variational state
vs = nk.vqs.MCState(sa, model, n_samples=2048)

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=H,
    optimizer=op,
    preconditioner=sr,
    variational_state=vs)

start = time.time()
gs.run(out='RBM', n_iter=n_iter)
end = time.time()

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')


  1%|          | 272/25000 [04:26<6:43:53,  1.02it/s, Energy=-15.414 ± 0.012 [σ²=0.262, R̂=1.0032]]   


KeyboardInterrupt: 

In [12]:
import netket as nk
import jax
from netket.operator.spin import sigmax,sigmaz
import time
hi = nk.hilbert.Spin(s=1/2, N=graph.n_nodes)
hi.random_state(key=jax.random.PRNGKey(0))

Array([-1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1.], dtype=float32)

In [ ]:
import jax
import netket as nk
from netket.operator.spin import sigmax,sigmaz
import time

N = 16

H = -1*sigmaz(hi,0)*sigmax(hi, 1)
H += -1*sigmax(hi,N-3)*sigmaz(hi, N-2)*sigmaz(hi, N-1)
H += -1*sigmax(hi,N-2)*sigmax(hi, N-1)
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmax(hi,i+2) for i in range(N-3)])

HX = -1 * sigmax(hi, 0) * sigmaz(hi, 1)
HX += -1 * sigmaz(hi, N - 3) * sigmax(hi, N - 2) * sigmax(hi, N - 1)
HX += -1 * sigmaz(hi, N - 2) * sigmaz(hi, N - 1)
HX += sum([-1 * sigmaz(hi, i) * sigmax(hi, (i + 1)) * sigmaz(hi, i + 2) for i in range(N - 3)])

graph = nk.graph.Chain(length=N, pbc=True)
g = nk.graph.Hypercube(length=N, n_dim=2, pbc=False)

ma = nk.models.RBM(alpha=2)
sa = nk.sampler.MetropolisLocal(hilbert=hi)

# Optimizer
op = nk.optimizer.Sgd(learning_rate=0.05)
# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.1)

# The variational state
vs = nk.vqs.MCState(sa, ma, n_samples=2048)

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=HX,
    optimizer=op,
    preconditioner=sr,
    variational_state=vs)

start = time.time()
gs.run(out='RBM', n_iter=25000)
end = time.time()

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')


In [37]:
L = 4
np.arange(L,L*(L-1), L)

array([4, 8])

In [38]:
np.arange(2*L-1, L*L-1, L )

array([ 7, 11])

In [39]:
(np.meshgrid(np.arange(L-2)+1, np.arange(L-2)+1)[1]*L+np.meshgrid(np.arange(L-2)+1, np.arange(L-2)+1)[0]).ravel()

array([ 5,  6,  9, 10])